# Lab 3 - Spatial Prediction Pipeline with ArcPy
## Weather
### Luke Zaruba
### GIS 5572: ArcGIS II
### 2023-04-06

In this notebook, an automated end-to-end pipeline is being implemented which identifies the best interpolation method based on accuracy, calculates point accuracy at locations with known values, aggregates the values to H3 hexagons (making it easier to work with in Postgres in GCP), before exporting the layers to the Postgres database on GCP.

In [1]:
# Import Packages
import sys

# Import WeatherLoader & Pipeline classes to run pipeline
# Need to Append Path for Pro - Can use '..' outside of Pro in IDE/Editor (if using structure of repo)
sys.path.append(r"C:\gitFiles\GIS5572\Lab3")
from utils.etl import WeatherLoader
from utils.interpolation import Pipeline

In [2]:
# Define Variables
output_gdb = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb"
output_directory = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3"
sde_connection = r"C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\lab3.sde"

## Clean and Aggregate Weather Data

In [3]:
# Create Weather Loader Object
weatherDataset = WeatherLoader(output_gdb, 6, 2022)

In [4]:
# Extract Weather Data
weatherDataset.extract()

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2022-06-01,69.619980,50.360012,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2022-06-01,65.840020,47.840023,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2022-06-01,69.080000,47.299988,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2022-06-01,66.919975,45.499990,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2022-06-01,69.980000,41.899990,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
2935,MN094,2022-06-30,NaN,NaN,NaN,I-35W at Mississippi I-35W Mile Post 19,-92.318176,46.669189
2936,MN095,2022-06-30,86.719980,54.499990,0.0,Blatnick Bridge South Abutment,-92.097100,46.734100
2937,MN096,2022-06-30,82.219980,66.560010,0.0,Albert Lea I-35 Mile Post 30,-93.277618,43.920101
2938,MN097,2022-06-30,82.219980,55.580000,0.0,Atkinson Bridge I-35 Mile Post 231,-92.531900,46.627800


In [5]:
# Transform Weather Data
weatherDataset.transform()

,station,date,max_tmpf,min_tmpf,precip,name,x,y
0,MN001,2022-06-01,69.619980,50.360012,0.0,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331
1,MN002,2022-06-01,65.840020,47.840023,0.0,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800
2,MN003,2022-06-01,69.080000,47.299988,0.0,Little Chicago I-35 Mile Post 70,-93.292427,44.478500
3,MN004,2022-06-01,66.919975,45.499990,0.0,Rush City I-35 Mile Post 157,-92.992752,45.642921
4,MN005,2022-06-01,69.980000,41.899990,0.0,Rutledge I-35 Mile Post 198,-92.838562,46.212570
...,...,...,...,...,...,...,...,...
2933,MN092,2022-06-30,78.619980,57.919980,0.0,Mitchell Bridge MN-169 Mile Post 343,-92.887600,47.445100
2936,MN095,2022-06-30,86.719980,54.499990,0.0,Blatnick Bridge South Abutment,-92.097100,46.734100
2937,MN096,2022-06-30,82.219980,66.560010,0.0,Albert Lea I-35 Mile Post 30,-93.277618,43.920101
2938,MN097,2022-06-30,82.219980,55.580000,0.0,Atkinson Bridge I-35 Mile Post 231,-92.531900,46.627800


In [6]:
# Aggregate Weather Data
weatherDataset.aggregate()

,station,name,x,y,max_tmpf,min_tmpf,precip
station,,,,,,,
MN001,MN001,Twin Lakes I-35 Mile Post 1,-93.354057,43.508331,80.348003,59.768001,0.000333
MN002,MN002,Silver Lake TH 7 Mile Post 1,-94.119100,44.906800,79.495998,59.491994,0.000333
MN003,MN003,Little Chicago I-35 Mile Post 70,-93.292427,44.478500,79.951999,58.057998,0.000000
MN004,MN004,Rush City I-35 Mile Post 157,-92.992752,45.642921,78.301997,55.826004,0.000333
MN005,MN005,Rutledge I-35 Mile Post 198,-92.838562,46.212570,80.539997,50.000002,0.000000
...,...,...,...,...,...,...,...
MN093,MN093,Silver Cliff MN-61 Mile Post 30,-91.591553,47.069969,55.040024,55.040024,0.000000
MN095,MN095,Blatnick Bridge South Abutment,-92.097100,46.734100,72.086002,51.260000,0.000000
MN096,MN096,Albert Lea I-35 Mile Post 30,-93.277618,43.920101,80.198001,59.186001,0.000000


In [7]:
# Load Weather Data to FC
weatherDataset.load()

## Run Pipeline

In [8]:
# Create Weather Pipeline Object - interpolates max temp in this case
weatherPipeline = Interpolator(weatherDataset.fc, output_directory, output_gdb, "max_tmpf")

In [9]:
# Run Exploratory Interpolation
weatherPipeline.run_exploratory_interpolation()

Start Time: Thursday, March 30, 2023 9:27:22 PM
Calculating Ordinary Kriging – Default
Calculating Ordinary Kriging – Optimized
Calculating Universal Kriging – Default
Calculating Universal Kriging – Optimized
Calculating Inverse Distance Weighted - Default
Calculating Inverse Distance Weighted - Optimized
 

--------------------------------------------
RANK | NAME
--------------------------------------------


1    | Ordinary Kriging – Optimized


2    | Ordinary Kriging – Default


3    | Inverse Distance Weighted - Optimized


4    | Inverse Distance Weighted - Default


5    | Universal Kriging – Optimized


6    | Universal Kriging – Default
--------------------------------------------
Succeeded at Thursday, March 30, 2023 9:27:26 PM (Elapsed Time: 3.33 seconds)


In [10]:
# Display Accuracy Assessment Table
weatherPipeline.display("DATAFRAME")

,OID_,DESCR,RANK,INCLUDED,RMSE,ME,ME_STD,RMSE_STD,ASE,MAX_ERROR,PERC_ERROR
0,1,Ordinary Kriging – Optimized,1,Yes,2.888571,-0.053954,-0.017746,0.940743,3.113293,15.911121,38.012951
1,2,Ordinary Kriging – Default,2,Yes,2.894638,-0.059246,-0.018237,0.921769,3.176252,16.093975,37.882756
2,3,Inverse Distance Weighted - Optimized,3,Yes,3.067512,0.183798,NaN,NaN,NaN,16.943615,34.172980
3,4,Inverse Distance Weighted - Default,4,Yes,3.242717,0.098952,NaN,NaN,NaN,16.022461,30.413183
4,5,Universal Kriging – Optimized,5,Yes,3.331589,-0.266408,-0.052783,0.933972,3.318572,16.570545,28.506041
5,6,Universal Kriging – Default,6,Yes,4.023262,-0.294120,-0.019946,0.970737,3.811392,25.645117,13.663136


In [11]:
# Create Point Accuracy Layer
weatherPipeline.create_point_accuracy_layer()

Point accuracy successfully generated at: C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb\aggMthWX_62022_point_diff


In [12]:
# Convert Interpolation Results to H3
weatherPipeline.convert_results_to_hex(contours=True)

Data successfully aggregated to H3 hexagons at: C:\Users\lukea\Documents\ArcGIS\Projects\GIS5572 - Lab 3\GIS5572 - Lab 3.gdb\aggMthWX_62022_h3_6


In [13]:
# Export H3 Hexagons to SDE
weatherPipeline.export_to_sde(sde_connection, "TESSELLATION")

In [14]:
# Export Point Accuracy Points to SDE
weatherPipeline.export_to_sde(sde_connection, "POINT_ACCURACY")